## Sources

In [1]:
import pandas as pd
import numpy as np
import csv
import uuid
from datetime import datetime

In [2]:
ROOT = "C:/Users/stual/PycharmProjects/peuplement_ontology/ontologie-peuplement/"
PATH = ROOT + "data/gentilly/SOURCES.csv"
OUTPUT_PATH = ROOT + "data/rdf/"

sources = pd.read_csv(PATH,header=0)
display(sources)

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/ontologie-peuplement/data/gentilly/SOURCES.csv'

In [ ]:
# Print the list of column headers
print(sources.columns.tolist())

['CODE_ARCHIVES', 'rdf:type', 'rico:name', 'rico:location', 'rico:identifier', 'rsfs:comment', 'isSourceType', 'hasCreationDate', 'hasValidTime/hasBeginning', 'hasValidTime/hasEnd', 'rico:isOrWasIncludedIn', 'rico:isOrWasInstantiationOf', 'rico:isOrWasDigitalInstantiationOf', 'rico:hasOrHadDerivedInstanciation', 'rico:hasOrHadPhysicalLocation', 'prov:wasGeneratedBy']


In [ ]:
from rdflib import Graph, Literal, Namespace, RDF, URIRef, BNode, PROV
from rdflib.namespace import XSD
from namespaces import *

# Create a new RDF graph
g = Graph()

g.bind('source', srcuri)
g.bind('cad', cad)
g.bind('add', add)
g.bind('rico', rico)
g.bind('fpo', fpo)
g.bind('time',time)
g.bind('srctype', srctype)
g.bind('mlclasse', mlclasse)
g.bind('activity', activity)

# Iterate over each row in the DataFrame
for index, row in sources.iterrows():
    CODE_ARCHIVES = str(int(row['CODE_ARCHIVES']))

    #URI = http://data.ign.fr/id/sources/{location}_{identifier}
    subject_uri = URIRef(srcuri + f"{CODE_ARCHIVES}_{str(row['rico:location'])}_{str(row['rico:identifier'])}")
    g.add((subject_uri, rico.identifier, Literal(row["rico:identifier"])))

    if not pd.isnull(row["rico:name"]):
        g.add((subject_uri, rico.name, Literal(row["rico:name"])))
    if not pd.isnull(row["isSourceType"]):
        g.add((subject_uri, cad.isSourceType, URIRef(srctype + f'{row["isSourceType"]}')))
        
    ##Define properties if object is a RecordResource
    if row["rdf:type"] == "RecordSet" or row["rdf:type"] == "Record":
        g.add((subject_uri, RDF.type, rico.RecordSet))
        g.add((subject_uri, rico.location, Literal(row["rico:location"])))
        
        if not pd.isnull(row["rico:isOrWasIncludedIn"]):
            g.add((subject_uri, rico.isOrWasIncludedIn, URIRef(srcuri + f"{CODE_ARCHIVES}_{str(row['rico:location'])}_{str(row['rico:isOrWasIncludedIn'])}")))
        ## Date de création
        creationDateBNode = BNode()
        g.add((subject_uri, rico.hasCreationDate, URIRef(creationDateBNode.n3())))
        g.add((URIRef(creationDateBNode.n3()), RDF.type, add.CrispTimeInstant))
        g.add((URIRef(creationDateBNode.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(creationDateBNode.n3()), add.timePrecision, time.Year))
        if '-' not in str(row["hasCreationDate"]):
            str_date = datetime.strptime(str(int(row["hasCreationDate"])) + '-01-01', '%Y-%m-%d')
        else:
            str_date = datetime.strptime(str(row["hasCreationDate"]), '%Y-%m-%d')
        g.add((URIRef(creationDateBNode.n3()), add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
        ## Temps valide
        validTimeNode = BNode()
        ### Début de validité
        validTimeNodeBegin = BNode()
        g.add((subject_uri, add.hasTime, URIRef(validTimeNode.n3())))
        g.add((URIRef(validTimeNode.n3()), RDF.type, time.Interval))
        g.add((URIRef(validTimeNode.n3()), add.hasBeginning, URIRef(validTimeNodeBegin.n3())))
        g.add((URIRef(validTimeNodeBegin.n3()), RDF.type, add.TimeInstant))
        g.add((URIRef(validTimeNodeBegin.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(validTimeNodeBegin.n3()), add.timePrecision, time.Year))
        if '-' not in str(row["hasValidTime/hasBeginning"]):
            str_date = datetime.strptime(str(int(row["hasValidTime/hasBeginning"])) + '-01-01', '%Y-%m-%d')
        else:
            str_date = datetime.strptime(str(row["hasValidTime/hasBeginning"]), '%Y-%m-%d')
        g.add((URIRef(validTimeNodeBegin.n3()), add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))
        ### Fin de validité
        validTimeNodeEnd = BNode()
        g.add((URIRef(validTimeNode.n3()), add.hasEnd, URIRef(validTimeNodeEnd.n3())))
        g.add((URIRef(validTimeNodeEnd.n3()), add.timeCalendar, time.Gregorian))
        g.add((URIRef(validTimeNodeEnd.n3()), add.timePrecision, time.Year))
        if '-' not in str(row["hasValidTime/hasEnd"]):
            str_date = datetime.strptime(str(int(row["hasValidTime/hasEnd"])) + '-12-31', '%Y-%m-%d')
        else:
            str_date = datetime.strptime(str(row["hasValidTime/hasEnd"]), '%Y-%m-%d')
        g.add((URIRef(validTimeNodeEnd.n3()), add.timeStamp, Literal(str_date, datatype=XSD.dateTimeStamp)))

    #else if Instanciation
    elif row["rdf:type"] == "Instanciation":
        if row["rico:identifier"][0:2] == '3P' or row["rico:identifier"][0:2] == 'CN':
            g.add((subject_uri, RDF.type, rico.Instantiation))
            g.add((subject_uri,rico.isOrWasInstantiationOf,URIRef(srcuri + f'{CODE_ARCHIVES}_{row["rico:location"]}_{row["rico:isOrWasInstantiationOf"]}')))
            for instanciation in row["rico:hasOrHadDerivedInstanciation"].split(';'):
                g.add((subject_uri, rico.hasOrHadDerivedInstanciation, URIRef(srcuri + f'{CODE_ARCHIVES}_{row["rico:location"]}_{instanciation}')))
            else:
                g.add((subject_uri, rico.hasOrHadDerivedInstanciation, URIRef(srcuri + f'{CODE_ARCHIVES}_{row["rico:location"]}_{row["rico:hasOrHadDerivedInstanciation"]}')))
            g.add((subject_uri,rico.hasOrHadPhysicalLocation,Literal(row['rico:hasOrHadPhysicalLocation'])))
        elif row["rico:identifier"][0:2] != '3P' or row["rico:identifier"][0:2] != 'CN' :
            g.add((subject_uri, RDF.type, rico.Instantiation))
            g.add((subject_uri,rico.isOrWasDigitalInstantiationOf,URIRef(srcuri + f'{CODE_ARCHIVES}_{row["rico:location"]}_{row["rico:isOrWasDigitalInstantiationOf"]}')))

    if not pd.isnull(row["prov:wasGeneratedBy"]):
        g.add((subject_uri, PROV.wasGeneratedBy, URIRef(activity + f'000{str(int(row["prov:wasGeneratedBy"]))}')))

# Print out the graph in Turtle syntax
print(g.serialize(format='turtle'))
#write the graph to a file
g.serialize(destination=f'{OUTPUT_PATH}/sources.ttl', format='turtle')

@prefix activity: <http://rdf.geohistoricaldata.org/id/codes/cadastre/activity/> .
@prefix add: <http://rdf.geohistoricaldata.org/def/address#> .
@prefix cad: <http://rdf.geohistoricaldata.org/def/cadastre#> .
@prefix prov: <http://www.w3.org/ns/prov#> .
@prefix rico: <https://www.ica.org/standards/RiC/ontology#> .
@prefix source: <http://rdf.geohistoricaldata.org/id/source/> .
@prefix srctype: <http://rdf.geohistoricaldata.org/id/codes/cadastre/sourceType/> .
@prefix time: <http://www.w3.org/2006/time#> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

source:75_Gentilly_CN80 a rico:Instantiation ;
    cad:isSourceType srctype:PlanParcellaireFinal ;
    rico:hasOrHadDerivedInstanciation source:75_Gentilly_CN80_digit ;
    rico:hasOrHadPhysicalLocation "Archives de Paris" ;
    rico:identifier "CN80" ;
    rico:isOrWasInstantiationOf source:75_Gentilly_PLAN_SD_1845 ;
    rico:name "Plan révisé - Section D dite de la Barrière de Fontainebleau" .

source:75_Gentilly_digit_CN80 a rico

<Graph identifier=N3de2ebdf4f524f7297f16214fc04b613 (<class 'rdflib.graph.Graph'>)>